# Exame Nacional do Ensino Médio (ENEM)

https://www.gov.br/inep/pt-br/areas-de-atuacao/avaliacao-e-exames-educacionais/enem/resultados <br>
https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem

<i>"Os microdados do Enem são o menor nível de desagregação de dados recolhidos por meio do exame. Eles atendem a demanda por informações específicas ao disponibilizar as provas, os gabaritos, as informações sobre os itens, as notas e o questionário respondido pelos inscritos no Enem.</i>"

In [1]:
#Importando pacotes e bibliotecas
import pandas as pd, numpy as np, matplotlib, matplotlib.pyplot as plt, gc, json

In [2]:
#Definindo funções auxiliares

def tic():
    import time
    global _start_time 
    _start_time = time.time()

def tac():
    import time
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))

def get_dir(year): return os.getcwd()+f'\\ENEM\\{year}\\DADOS\\MICRODADOS_ENEM_{year}.csv'

def save_dir(year):
    dirs = {'csv':os.getcwd()+f'\\ENEM\\CONSOLIDADO\\MICRODADOS_ENEM_CONSOLIDADO_{year}.csv',
            'json':os.getcwd()+f'\\ENEM\\CONSOLIDADO\\STATS_{year}.json'}
    return dirs

def check_value(df, col, vls = [0, 1, 2, 3, 4, 5]): return sum(~df[col].copy().fillna(df[col][~df[col].isna()].min()).isin(vls)) == 0

def check_type(df, col, tp): return df[col].copy().fillna(df[col][~df[col].isna()].min()).astype(tp).dtype == tp

def check_size(df, col, tp, minimo, maximo):
    if tp in ['float', 'float64']:
        ck = df[col].copy().fillna(df[col][~df[col].isna()].min()).astype('str').str.replace('.','').str.len()
    elif tp in ['int', 'int64']:
        ck = df[col].copy().fillna(df[col][~df[col].isna()].min()).astype('int').astype('str').str.len()
    elif tp in ['str', 'O']:
        ck = df[col].copy().fillna(df[col][~df[col].isna()].min()).str.len()
    else:
        return 'ERRO'
    return sum(~ck.isin(range(minimo, maximo+1))) == 0

def check_uniqueness(df, col): return len(df[col].copy().unique()) == df.shape[0]

def check_nulls(df, col): return sum(df[col].isna())

def print_results(col, results):
    print(col); print('-'*25)
    tm = max([len(k) for k in results.keys()])
    for k,v in zip(results.keys(), results.values()):
        nk = k+':'+(tm - len(k))*' '
        if (type(v) == type(True)) and not v:
            print(f'{nk}\t\033[91m{v}\033[0m')
        else:
            print(f'{nk}\t{v}')

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def validate(df, col, year, get_values = False, time = False):
    
    if time: tic()
    
    results = {}
    values = False
    types = False
    sizes = False
    uniques = False
    nulls = False
    tp, vls, minimo, maximo = '', [], 0, 0
       
    if col == 'NU_INSCRICAO':
        types = True
        sizes = True
        uniques = True
        nulls = True
        tp, minimo, maximo = 'int', 0, 12
    elif col == 'NU_ANO':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [year], 0, 4
    elif col == 'CO_MUNICIPIO_RESIDENCIA':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'int', 0, 7
    elif col == 'NO_MUNICIPIO_RESIDENCIA':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'O', 0, 150
    elif col == 'CO_UF_RESIDENCIA':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'int', 0, 2
    elif col == 'SG_UF_RESIDENCIA':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'O', 2, 2
    elif col == 'NU_IDADE':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', range(10, 101), 2, 3
    elif col == 'TP_SEXO':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'O', ['M', 'F'], 1, 1
    elif col == 'TP_ESTADO_CIVIL':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1, 2, 3, 4], 1, 1
    elif col == 'TP_COR_RACA':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1, 2, 3, 4, 5], 1, 1
    elif col == 'TP_ST_CONCLUSAO':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [1, 2, 3, 4], 1, 1
    elif col == 'TP_ENSINO':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [1, 2, 3, 4], 1, 1
    elif col == 'CO_ESCOLA':
        types = True
        sizes = True        
        nulls = True
        tp, minimo, maximo = 'int', 0, 8
    elif col == 'CO_MUNICIPIO_ESC':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'int', 0, 7
    elif col == 'NO_MUNICIPIO_ESC':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'O', 0, 150
    elif col == 'CO_UF_ESC':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'int', 0, 2
    elif col == 'SG_UF_ESC':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'O', 2, 2
    elif col == 'TP_DEPENDENCIA_ADM_ESC':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [1, 2, 3, 4], 1, 1
    elif col == 'TP_LOCALIZACAO_ESC':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [1, 2], 1, 1
    elif col == 'TP_SIT_FUNC_ESC':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [1, 2, 3, 4], 1, 1
    elif col == 'IN_BAIXA_VISAO':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1
    elif col == 'IN_CEGUEIRA':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1
    elif col == 'IN_SURDEZ':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1 
    elif col == 'IN_DEFICIENCIA_AUDITIVA':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1
    elif col == 'IN_DEFICIENCIA_FISICA':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1
    elif col == 'IN_DEFICIENCIA_MENTAL':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1
    elif col == 'IN_DEFICIT_ATENCAO':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1
    elif col == 'IN_DISLEXIA':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1
    elif col == 'IN_GESTANTE':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1
    elif col == 'IN_LACTANTE':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1
    elif col == 'CO_MUNICIPIO_PROVA':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'int', 0, 7
    elif col == 'NO_MUNICIPIO_PROVA':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'O', 0, 150
    elif col == 'CO_UF_PROVA':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'int', 0, 2
    elif col == 'SG_UF_PROVA':
        types = True
        sizes = True
        nulls = True
        tp, minimo, maximo = 'O', 2, 2
    elif col == 'TP_PRESENCA_CN':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1, 2], 1, 1
    elif col == 'TP_PRESENCA_CH':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1, 2], 1, 1
    elif col == 'TP_PRESENCA_LC':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1, 2], 1, 1
    elif col == 'TP_PRESENCA_MT':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1, 2], 1, 1
    elif col == 'NU_NOTA_CN':
        types = True
        sizes = True       
        nulls = True
        tp, minimo, maximo = 'float', 0, 9
    elif col == 'NU_NOTA_CH':
        types = True
        sizes = True       
        nulls = True
        tp, minimo, maximo = 'float', 0, 9
    elif col == 'NU_NOTA_LC':
            types = True
            sizes = True       
            nulls = True
            tp, minimo, maximo = 'float', 0, 9
    elif col == 'NU_NOTA_MT':
            types = True
            sizes = True       
            nulls = True
            tp, minimo, maximo = 'float', 0, 9
    elif col == 'TP_LINGUA':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [0, 1], 1, 1
    elif col == 'TP_STATUS_REDACAO':
        types = True
        sizes = True
        values = True
        nulls = True
        tp, vls, minimo, maximo = 'int', [1, 2, 3, 4, 5, 6, 7, 8, 9, 98], 1, 2
    elif col == 'NU_NOTA_COMP1':
            types = True
            sizes = True       
            nulls = True
            tp, minimo, maximo = 'float', 0, 9
    elif col == 'NU_NOTA_COMP2':
            types = True
            sizes = True       
            nulls = True
            tp, minimo, maximo = 'float', 0, 9
    elif col == 'NU_NOTA_COMP3':
            types = True
            sizes = True       
            nulls = True
            tp, minimo, maximo = 'float', 0, 9
    elif col == 'NU_NOTA_COMP4':
            types = True
            sizes = True       
            nulls = True
            tp, minimo, maximo = 'float', 0, 9
    elif col == 'NU_NOTA_COMP5':
            types = True
            sizes = True       
            nulls = True
            tp, minimo, maximo = 'float', 0, 9
    elif col == 'NU_NOTA_REDACAO':
            types = True
            sizes = True       
            nulls = True
            tp, minimo, maximo = 'float', 0, 9
    else:
        return None
   
    if types: results['Tipo'] = check_type(df, col, tp)
    if sizes: results['Tamanho'] = check_size(df, col, tp, minimo, maximo)
    if values: results['Valores'] = check_value(df, col, vls)
    if uniques: results['Unicidade'] = check_uniqueness(df, col)
    if nulls: results['Nulos'] = check_nulls(df, col)
        
        
    if get_values:
        
        cols_min_avg_max = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
                        'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']
        cols_none = ['NU_INSCRICAO', 'NU_ANO']
        
        if col in cols_none:
            res_values = {'nul':df[col].isna().sum()}
        elif col in cols_min_avg_max:
            res_values = {'min':df[col].min(), 'avg':df[col].mean(), 'max':df[col].max(), 'nul':df[col].isna().sum()}
        else:
            vdict = dict(df[col].value_counts(dropna=False).sort_index())
            #vdict = {str(k):v for k, v in zip(vdict.keys(), vdict.values())}
            res_values = {'val':vdict, 'nul':df[col].isna().sum()}
        
    if time: tac()
    
    if get_values:
        return results, res_values
    else:   
        return results

def check_municipios(df):
    kds = 'RESIDENCIA', 'ESC', 'PROVA'
    err_dict = {}
    for kd in kds:
        col_co, col_no = f'CO_MUNICIPIO_{kd}', f'NO_MUNICIPIO_{kd}'
        check = ~df[col_co].copy().isna()
        errs = df[check][[col_co, col_no]][~(df[check][col_co].map(dict_municipios)==df[check][col_no])].copy()
        errs['COL'] = kd; errs = errs[['COL', col_co, col_no]].copy()
        errs['NOME_ESPERADO'] = errs[col_co].map(dict_municipios)
        errs.columns = ['COL', 'CO_MUNICIPIO', 'NO_MUNICIPIO', 'NOME_ESPERADO']
        err_dict[kd] = errs[~errs.duplicated()].reset_index(drop = True)
    errs = err_dict['RESIDENCIA'].append(err_dict['ESC'], ignore_index = True).append(err_dict['PROVA'], ignore_index = True)
    return errs

def corrige_municipio(df_erros):
    for i in range(df_erros.shape[0]):
        col_co = 'CO_MUNICIPIO_' + df_erros.iloc[i]['COL']
        col_no = 'NO_MUNICIPIO_' + df_erros.iloc[i]['COL']
        co = df_erros.iloc[i]['CO_MUNICIPIO']
        va = df_erros.iloc[i]['NOME_ESPERADO']
        df.loc[df[col_co]==co, col_no] = va

def check_ufs(df):
    kds = 'RESIDENCIA', 'ESC', 'PROVA'
    err_dict = {}
    for kd in kds:
        col_co, col_no = f'CO_UF_{kd}', f'SG_UF_{kd}'
        check = ~df[col_co].copy().isna()
        errs = df[check][[col_co, col_no]][~(df[check][col_co].map(dict_ufs)==df[check][col_no])].copy()
        errs['COL'] = kd; errs = errs[['COL', col_co, col_no]].copy()
        errs['SIGLA_ESPERADA'] = errs[col_co].map(dict_ufs)
        errs.columns = ['COL', 'CO_MUNICIPIO', 'SG_MUNICIPIO', 'SIGLA_ESPERADA']
        err_dict[kd] = errs[~errs.duplicated()].reset_index(drop = True)
    errs = err_dict['RESIDENCIA'].append(err_dict['ESC'], ignore_index = True).append(err_dict['PROVA'], ignore_index = True)
    return errs

In [3]:
#Estabelecendo os campos de interese (com base no dicionário, tendo como referencia os dados de 2019)
campos = ['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA', 'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
          'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_ST_CONCLUSAO', 'TP_ENSINO', 'CO_ESCOLA', 'CO_MUNICIPIO_ESC',
          'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
          'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA', 'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL',
          'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_GESTANTE', 'IN_LACTANTE', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA',
          'CO_UF_PROVA', 'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'NU_NOTA_CN',
          'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
          'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']

## 2019

In [5]:
#Obtendo informações do ano
year = 2019
tic()
df = pd.read_csv(get_dir(year), sep = ';', encoding = 'latin')
tac()

Time passed: 0hour:0min:54sec


In [5]:
#selecionando apenas os campos de interesse
df = df[campos]
print(df.shape)

(5095270, 50)


In [451]:
#Verificando a consistência dos dados e sumarizando estatísticas descritivas
year_res_dict = {year:{'cols':df.shape[1], 'rows':df.shape[0], 'data':{}}}
tic()
for col in campos:
    res_prt, res_val = validate(df, col, year, get_values=True)
    print_results(col, res_prt); print()
    year_res_dict[year]['data'][col] = res_val
tac()

NU_INSCRICAO
-------------------------
Tipo:     	True
Tamanho:  	True
Unicidade:	True
Nulos:    	0

NU_ANO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

CO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

CO_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

SG_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NU_IDADE
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	69

TP_SEXO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ESTADO_CIVIL
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_COR_RACA
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ST_CONCLUSAO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_E

In [469]:
#Armazenando as estatísticas descritivas sumarizadas em um arquivo json
with open(save_dir(year)['json'], 'w', encoding='utf-8') as fp: json.dump(year_res_dict, fp, cls=NpEncoder, ensure_ascii=False)

In [470]:
#Armazenando os dados do ano em um arquivo csv
tic()
df.to_csv(save_dir(year)['csv'], sep=';', header = True, index = False, encoding='latin')
tac()

Time passed: 0hour:1min:23sec


In [6]:
#Definindo dicionarios para municipios
print('dict_municipios')
print('-'*25)
municipio_dict1 = dict(df[['CO_MUNICIPIO_RESIDENCIA',
                           'NO_MUNICIPIO_RESIDENCIA']][~df[['CO_MUNICIPIO_RESIDENCIA',
                                                            'NO_MUNICIPIO_RESIDENCIA']].duplicated()].values)
print('Dicionário Municipios (Res):', len(set(municipio_dict1.keys())) == len(df['CO_MUNICIPIO_RESIDENCIA'].unique()))
municipio_dict2 = dict(df[['CO_MUNICIPIO_ESC',
                            'CO_MUNICIPIO_ESC']][~df[['CO_MUNICIPIO_ESC',
                                                      'CO_MUNICIPIO_ESC']].duplicated()].values)
municipio_dict2 = {int(k):v for k, v in zip(municipio_dict2.keys(), municipio_dict2.values()) if not np.isnan(k)}

print('Dicionário Municipios (Esc):', len(set(municipio_dict2.keys()))==len(df[~df['CO_MUNICIPIO_ESC'].isna()]['CO_MUNICIPIO_ESC'].unique()))
municipio_dict3 = dict(df[['CO_MUNICIPIO_PROVA',
                            'CO_MUNICIPIO_PROVA']][~df[['CO_MUNICIPIO_PROVA',
                                                      'CO_MUNICIPIO_PROVA']].duplicated()].values)
municipio_dict3 = {int(k):v for k, v in zip(municipio_dict3.keys(), municipio_dict3.values()) if not np.isnan(k)}

print('Dicionário Municipios (Pro):', len(set(municipio_dict3.keys()))==len(df[~df['CO_MUNICIPIO_PROVA'].isna()]['CO_MUNICIPIO_PROVA'].unique()))

#Verificando se o dicionario de municipios gerado a partir dos campos CO/NO_MUNICIPIO_RESIDENCIA engloba os demais
val = sum(~pd.Series(municipio_dict2.keys()).isin(municipio_dict1.keys())) + sum(~pd.Series(municipio_dict3.keys()).isin(municipio_dict1.keys()))
print('Dicionário Municípios (Final):', val==0)

#Definindo um dicionário final
dict_municipios = municipio_dict1

print()

#verificando aplicação do dicionario nas colunas de municipio
check = ~df['CO_MUNICIPIO_ESC'].isna()
print('Verificação dicionário (Esc):', sum(~(df[check]['CO_MUNICIPIO_ESC'].map(dict_municipios) == df[check]['NO_MUNICIPIO_ESC'])) == 0)

check = ~df['CO_MUNICIPIO_PROVA'].isna()
print('Verificação dicionário (Pro):', sum(~(df[check]['CO_MUNICIPIO_PROVA'].map(dict_municipios) == df[check]['NO_MUNICIPIO_PROVA'])) == 0)

dict_municipios
-------------------------
Dicionário Municipios (Res): True
Dicionário Municipios (Esc): True
Dicionário Municipios (Pro): True
Dicionário Municípios (Final): True

Verificação dicionário (Esc): True
Verificação dicionário (Pro): True


In [7]:
#Definindo um dicionario para UFs
print('dict_ufs')
print('-'*25)
dict_ufs = dict(df[['CO_UF_RESIDENCIA','SG_UF_RESIDENCIA']][~df[['CO_UF_RESIDENCIA','SG_UF_RESIDENCIA']].duplicated()].values)
check = ~df['CO_UF_RESIDENCIA'].isna()
print('Dicionário UFs (Res):', sum(~(df[check]['CO_UF_RESIDENCIA'].map(dict_ufs) == df[check]['SG_UF_RESIDENCIA'])) == 0)
check = ~df['CO_UF_ESC'].isna()
print('Dicionário UFs (Esc):', sum(~(df[check]['CO_UF_ESC'].map(dict_ufs) == df[check]['SG_UF_ESC'])) == 0)
check = ~df['CO_UF_PROVA'].isna()
print('Dicionário UFs (Pro):', sum(~(df[check]['CO_UF_PROVA'].map(dict_ufs) == df[check]['SG_UF_PROVA'])) == 0)
print('Dicionário UFs (Final):', True)

dict_ufs
-------------------------
Dicionário UFs (Res): True
Dicionário UFs (Esc): True
Dicionário UFs (Pro): True
Dicionário UFs (Final): True


In [8]:
#Armazenando os dicionarios em arquivos json

with open(os.getcwd()+f'\\ENEM\\CONSOLIDADO\\dict_municipios.json', 'w', encoding='utf-8') as fp: json.dump(dict_municipios,
                                                                                                            fp, cls=NpEncoder,
                                                                                                            ensure_ascii=False)
with open(os.getcwd()+f'\\ENEM\\CONSOLIDADO\\dict_ufs.json', 'w', encoding='utf-8') as fp: json.dump(dict_ufs,
                                                                                                     fp, cls=NpEncoder,
                                                                                                     ensure_ascii=False)

In [4]:
#Obter dicionários
rood_dir = os.getcwd()+'\\ENEM\\CONSOLIDADO\\'
with open(rood_dir+'dict_municipios.json', encoding='utf-8') as json_file: dict_municipios = json.load(json_file)
with open(rood_dir+'dict_ufs.json', encoding='utf-8') as json_file: dict_ufs = json.load(json_file)
dict_municipios = {int(k):v for k, v in zip(dict_municipios.keys(), dict_municipios.values())}
dict_ufs = {int(k):v for k, v in zip(dict_ufs.keys(), dict_ufs.values())}
dict_ufs_inv = {v:k for v, k in zip(dict_ufs.values(), dict_ufs.keys())}

___

## 2018

In [601]:
#Obtendo informações do ano
year = 2018
tic()
df = pd.read_csv(get_dir(year), sep = ';', encoding = 'latin')
tac()

Time passed: 0hour:1min:34sec


In [602]:
#Ajustando dados: TP_ESTADO_CIVIL
#--------------------------------------------------------------------------------------
#NaN                                       → 0 Não informado 
#0 Solteiro(a)                             → 1 Solteiro(a)
#1 Casado(a)/Mora com companheiro(a)       → 2 Casado(a)/Mora com companheiro(a)
#2 Divorciado(a)/Desquitado(a)/Separado(a) → 3 Divorciado(a)/Desquitado(a)/Separado(a)
#3 Viúvo(a)                                → 4 Viúvo(a)
#--------------------------------------------------------------------------------------
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())
df['TP_ESTADO_CIVIL'] = df['TP_ESTADO_CIVIL'].map({0:1, 1:2, 2:3, 3:4})
df['TP_ESTADO_CIVIL'].fillna(0, inplace = True)
print();print(50*'-');print()
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())

0.0    4748654
1.0     468245
2.0      71890
3.0       7321
NaN     217637
Name: TP_ESTADO_CIVIL, dtype: int64

--------------------------------------------------

0.0     217637
1.0    4748654
2.0     468245
3.0      71890
4.0       7321
Name: TP_ESTADO_CIVIL, dtype: int64


In [603]:
#Ajustando dados: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
display(check_municipios(df))
df.loc[df['CO_MUNICIPIO_RESIDENCIA']==3550001, 'NO_MUNICIPIO_RESIDENCIA'] = dict_municipios[3550001]
df.loc[df['CO_MUNICIPIO_ESC']==3550001, 'NO_MUNICIPIO_ESC'] = dict_municipios[3550001]
print();print(50*'-');print()
display(check_municipios(df))

,COL,CO_MUNICIPIO,NO_MUNICIPIO
0,RESIDENCIA,3550001.0,São Luís do Paraitinga
1,ESC,3550001.0,São Luís do Paraitinga


,COL,CO_MUNICIPIO,NO_MUNICIPIO


In [604]:
#Ajustando dados: SG_UF_RESIDENCIA/ESC/PROVA
check_ufs(df)

,COL,CO_MUNICIPIO,SG_MUNICIPIO


In [605]:
#selecionando apenas os campos de interesse
df = df[campos]
print(df.shape)

(5513747, 50)


In [606]:
#Verificando a consistência dos dados e sumarizando estatísticas descritivas
year_res_dict = {year:{'cols':df.shape[1], 'rows':df.shape[0], 'data':{}}}
tic()
for col in campos:
    res_prt, res_val = validate(df, col, year, get_values=True)
    print_results(col, res_prt); print()
    year_res_dict[year]['data'][col] = res_val
tac()

NU_INSCRICAO
-------------------------
Tipo:     	True
Tamanho:  	True
Unicidade:	True
Nulos:    	0

NU_ANO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

CO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

CO_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

SG_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NU_IDADE
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	97

TP_SEXO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ESTADO_CIVIL
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_COR_RACA
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ST_CONCLUSAO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_E

In [608]:
#Armazenando as estatísticas descritivas sumarizadas em um arquivo json
with open(save_dir(year)['json'], 'w', encoding='utf-8') as fp: json.dump(year_res_dict, fp, cls=NpEncoder, ensure_ascii=False)

In [609]:
#Armazenando os dados do ano em um arquivo csv
tic()
df.to_csv(save_dir(year)['csv'], sep=';', header = True, index = False, encoding='latin')
tac()

Time passed: 0hour:1min:45sec


___

## 2017

In [677]:
#Obtendo informações do ano
year = 2017
tic()
df = pd.read_csv(get_dir(year), sep = ';', encoding = 'latin')
tac()

Time passed: 0hour:2min:40sec


In [678]:
#Ajustando dados 1: TP_ESTADO_CIVIL
#--------------------------------------------------------------------------------------
#NaN                                       → 0 Não informado 
#0 Solteiro(a)                             → 1 Solteiro(a)
#1 Casado(a)/Mora com companheiro(a)       → 2 Casado(a)/Mora com companheiro(a)
#2 Divorciado(a)/Desquitado(a)/Separado(a) → 3 Divorciado(a)/Desquitado(a)/Separado(a)
#3 Viúvo(a)                                → 4 Viúvo(a)
#--------------------------------------------------------------------------------------
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())
df['TP_ESTADO_CIVIL'] = df['TP_ESTADO_CIVIL'].map({0:1, 1:2, 2:3, 3:4})
df['TP_ESTADO_CIVIL'].fillna(0, inplace = True)
print();print(50*'-');print()
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())

0.0    5680562
1.0     669750
2.0      98879
3.0      10509
NaN     271641
Name: TP_ESTADO_CIVIL, dtype: int64

--------------------------------------------------

0.0     271641
1.0    5680562
2.0     669750
3.0      98879
4.0      10509
Name: TP_ESTADO_CIVIL, dtype: int64


In [679]:
#Selecionando apenas os campos de interesse
df = df[campos].copy()
print(df.shape)

(6731341, 50)


In [680]:
#Ajustando dados 2: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO
0,RESIDENCIA,2917334.0,Iuiú,Iuiu
1,RESIDENCIA,3530805.0,MOGI MIRIM,Mogi Mirim
2,RESIDENCIA,4315503.0,Restinga Seca,Restinga Sêca
3,RESIDENCIA,2306306.0,Itapagé,Itapajé
4,RESIDENCIA,3506607.0,Biritiba-Mirim,Biritiba Mirim
5,RESIDENCIA,2902054.0,Araças,Araçás
6,RESIDENCIA,3147808.0,Passa-Vinte,Passa Vinte
7,RESIDENCIA,3550001.0,São Luís do Paraitinga,São Luiz do Paraitinga
8,RESIDENCIA,4209607.0,Lauro Muller,Lauro Müller
9,RESIDENCIA,2613800.0,São Vicente Ferrer,São Vicente Férrer


In [681]:
#Ajustando dados 2: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
corrige_municipio(df_erros_municipio)

In [682]:
#Ajustando dados 2: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO


In [683]:
#Ajustando dados: SG_UF_RESIDENCIA/ESC/PROVA
check_ufs(df)

,COL,CO_MUNICIPIO,SG_MUNICIPIO,SIGLA_ESPERADA


In [684]:
#Verificando a consistência dos dados e sumarizando estatísticas descritivas
year_res_dict = {year:{'cols':df.shape[1], 'rows':df.shape[0], 'data':{}}}
tic()
for col in campos:
    res_prt, res_val = validate(df, col, year, get_values=True)
    print_results(col, res_prt); print()
    year_res_dict[year]['data'][col] = res_val
tac()

NU_INSCRICAO
-------------------------
Tipo:     	True
Tamanho:  	True
Unicidade:	True
Nulos:    	0

NU_ANO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

CO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

CO_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

SG_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NU_IDADE
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	101

TP_SEXO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ESTADO_CIVIL
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_COR_RACA
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ST_CONCLUSAO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_

In [685]:
#Armazenando as estatísticas descritivas sumarizadas em um arquivo json
with open(save_dir(year)['json'], 'w', encoding='utf-8') as fp: json.dump(year_res_dict, fp, cls=NpEncoder, ensure_ascii=False)

In [686]:
#Armazenando os dados do ano em um arquivo csv
tic()
df.to_csv(save_dir(year)['csv'], sep=';', header = True, index = False, encoding='latin')
tac()

Time passed: 0hour:2min:46sec


___

## 2016

In [5]:
#Obtendo informações do ano
year = 2016
tic()
df = pd.read_csv(get_dir(year), sep = ';', encoding = 'latin')
tac()

Time passed: 0hour:3min:21sec


In [6]:
#Ajustando dados 1: TP_ESTADO_CIVIL
#--------------------------------------------------------------------------------------
#NaN                                       → 0 Não informado 
#0 Solteiro(a)                             → 1 Solteiro(a)
#1 Casado(a)/Mora com companheiro(a)       → 2 Casado(a)/Mora com companheiro(a)
#2 Divorciado(a)/Desquitado(a)/Separado(a) → 3 Divorciado(a)/Desquitado(a)/Separado(a)
#3 Viúvo(a)                                → 4 Viúvo(a)
#--------------------------------------------------------------------------------------
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())
df['TP_ESTADO_CIVIL'] = df['TP_ESTADO_CIVIL'].map({0:1, 1:2, 2:3, 3:4})
df['TP_ESTADO_CIVIL'].fillna(0, inplace = True)
print();print(50*'-');print()
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())

0.0    7144455
1.0     947502
2.0     138765
3.0      16104
NaN     380541
Name: TP_ESTADO_CIVIL, dtype: int64

--------------------------------------------------

0.0     380541
1.0    7144455
2.0     947502
3.0     138765
4.0      16104
Name: TP_ESTADO_CIVIL, dtype: int64


In [8]:
#Ajustando dados 2: TP_COR_RACA
#--------------------------------------------------------------------------------------
#6 Não dispõe da informação → 0 Não declarado
#--------------------------------------------------------------------------------------
print(df['TP_COR_RACA'].value_counts(dropna = False).sort_index())
df['TP_COR_RACA'] = df['TP_COR_RACA'].map({0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:0})
print();print(50*'-');print()
print(df['TP_COR_RACA'].value_counts(dropna = False).sort_index())

0     164629
1    3047482
2    1155766
3    4003922
4     201102
5      54253
6        213
Name: TP_COR_RACA, dtype: int64

--------------------------------------------------

0     164842
1    3047482
2    1155766
3    4003922
4     201102
5      54253
Name: TP_COR_RACA, dtype: int64


In [11]:
#Selecionando apenas os campos de interesse
df = df[campos].copy()
print(df.shape)

(8627367, 50)


In [12]:
#Ajustando dados 3: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO
0,RESIDENCIA,3530805.0,Moji Mirim,Mogi Mirim
1,RESIDENCIA,4209607.0,Lauro Muller,Lauro Müller
2,RESIDENCIA,3506607.0,Biritiba-Mirim,Biritiba Mirim
3,RESIDENCIA,2306306.0,Itapagé,Itapajé
4,RESIDENCIA,3200706.0,Atilio Vivacqua,Atílio Vivacqua
5,RESIDENCIA,2613800.0,São Vicente Ferrer,São Vicente Férrer
6,RESIDENCIA,2922250.0,Muquém de São Francisco,Muquém do São Francisco
7,RESIDENCIA,5220157.0,São Luíz do Norte,São Luiz do Norte
8,RESIDENCIA,2902054.0,Araças,Araçás
9,RESIDENCIA,3150539.0,Pingo-d'Água,Pingo d'Água


In [13]:
#Ajustando dados 3: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
corrige_municipio(df_erros_municipio)

In [14]:
#Ajustando dados 3: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO


In [15]:
#Ajustando dados: SG_UF_RESIDENCIA/ESC/PROVA
check_ufs(df)

,COL,CO_MUNICIPIO,SG_MUNICIPIO,SIGLA_ESPERADA


In [16]:
#Verificando a consistência dos dados e sumarizando estatísticas descritivas
year_res_dict = {year:{'cols':df.shape[1], 'rows':df.shape[0], 'data':{}}}
tic()
for col in campos:
    res_prt, res_val = validate(df, col, year, get_values=True)
    print_results(col, res_prt); print()
    year_res_dict[year]['data'][col] = res_val
tac()

NU_INSCRICAO
-------------------------
Tipo:     	True
Tamanho:  	True
Unicidade:	True
Nulos:    	0

NU_ANO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

CO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

CO_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

SG_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NU_IDADE
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	102

TP_SEXO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ESTADO_CIVIL
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_COR_RACA
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ST_CONCLUSAO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_

In [17]:
#Armazenando as estatísticas descritivas sumarizadas em um arquivo json
with open(save_dir(year)['json'], 'w', encoding='utf-8') as fp: json.dump(year_res_dict, fp, cls=NpEncoder, ensure_ascii=False)

In [18]:
#Armazenando os dados do ano em um arquivo csv
tic()
df.to_csv(save_dir(year)['csv'], sep=';', header = True, index = False, encoding='latin')
tac()

Time passed: 0hour:3min:4sec


___

## 2015

In [30]:
#Obtendo informações do ano
year = 2015
tic()
df = pd.read_csv(get_dir(year), sep = ',', encoding = 'latin') #separador mudou de ';' para ','
tac()

Time passed: 0hour:4min:1sec


In [36]:
#Ajustando dados 1: TP_ESTADO_CIVIL
#--------------------------------------------------------------------------------------
#NaN                                       → 0 Não informado 
#0 Solteiro(a)                             → 1 Solteiro(a)
#1 Casado(a)/Mora com companheiro(a)       → 2 Casado(a)/Mora com companheiro(a)
#2 Divorciado(a)/Desquitado(a)/Separado(a) → 3 Divorciado(a)/Desquitado(a)/Separado(a)
#3 Viúvo(a)                                → 4 Viúvo(a)
#--------------------------------------------------------------------------------------
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())
df['TP_ESTADO_CIVIL'] = df['TP_ESTADO_CIVIL'].map({0:1, 1:2, 2:3, 3:4})
df['TP_ESTADO_CIVIL'].fillna(0, inplace = True)
print();print(50*'-');print()
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())

0.0    6417079
1.0     866112
2.0     124427
3.0      15577
NaN     323232
Name: TP_ESTADO_CIVIL, dtype: int64

--------------------------------------------------

0.0     323232
1.0    6417079
2.0     866112
3.0     124427
4.0      15577
Name: TP_ESTADO_CIVIL, dtype: int64


In [37]:
#Ajustando dados 2: TP_COR_RACA
#--------------------------------------------------------------------------------------
#6 Não dispõe da informação → 0 Não declarado
#--------------------------------------------------------------------------------------
print(df['TP_COR_RACA'].value_counts(dropna = False).sort_index())
df['TP_COR_RACA'] = df['TP_COR_RACA'].map({0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:0})
print();print(50*'-');print()
print(df['TP_COR_RACA'].value_counts(dropna = False).sort_index())

0     124527
1    2843817
2    1013917
3    3528864
4     170228
5      44806
6      20268
Name: TP_COR_RACA, dtype: int64

--------------------------------------------------

0     144795
1    2843817
2    1013917
3    3528864
4     170228
5      44806
Name: TP_COR_RACA, dtype: int64


In [38]:
#Selecionando apenas os campos de interesse
df = df[campos].copy()
print(df.shape)

(7746427, 50)


In [39]:
#Ajustando dados 3: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO
0,RESIDENCIA,3200706.0,Atilio Vivacqua,Atílio Vivacqua
1,RESIDENCIA,2306306.0,Itapagé,Itapajé
2,RESIDENCIA,3506607.0,Biritiba-Mirim,Biritiba Mirim
3,RESIDENCIA,4209607.0,Lauro Muller,Lauro Müller
4,RESIDENCIA,2917334.0,Iuiú,Iuiu
5,RESIDENCIA,3530805.0,Moji Mirim,Mogi Mirim
6,RESIDENCIA,2902054.0,Araças,Araçás
7,RESIDENCIA,2408409.0,Olho-d'Água do Borges,Olho d'Água do Borges
8,RESIDENCIA,3550001.0,São Luís do Paraitinga,São Luiz do Paraitinga
9,RESIDENCIA,1502954.0,Eldorado dos Carajás,Eldorado do Carajás


In [40]:
#Ajustando dados 3: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
corrige_municipio(df_erros_municipio)

In [41]:
#Ajustando dados 3: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO


In [42]:
#Ajustando dados: SG_UF_RESIDENCIA/ESC/PROVA
check_ufs(df)

,COL,CO_MUNICIPIO,SG_MUNICIPIO,SIGLA_ESPERADA


In [43]:
#Verificando a consistência dos dados e sumarizando estatísticas descritivas
year_res_dict = {year:{'cols':df.shape[1], 'rows':df.shape[0], 'data':{}}}
tic()
for col in campos:
    res_prt, res_val = validate(df, col, year, get_values=True)
    print_results(col, res_prt); print()
    year_res_dict[year]['data'][col] = res_val
tac()

NU_INSCRICAO
-------------------------
Tipo:     	True
Tamanho:  	True
Unicidade:	True
Nulos:    	0

NU_ANO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

CO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

CO_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

SG_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NU_IDADE
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	75

TP_SEXO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ESTADO_CIVIL
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_COR_RACA
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ST_CONCLUSAO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_E

In [44]:
#Armazenando as estatísticas descritivas sumarizadas em um arquivo json
with open(save_dir(year)['json'], 'w', encoding='utf-8') as fp: json.dump(year_res_dict, fp, cls=NpEncoder, ensure_ascii=False)

In [45]:
#Armazenando os dados do ano em um arquivo csv
tic()
df.to_csv(save_dir(year)['csv'], sep=';', header = True, index = False, encoding='latin')
tac()

Time passed: 0hour:2min:43sec


___

## 2014

In [5]:
#Obtendo informações do ano
year = 2014
tic()
df = pd.read_csv(get_dir(year), sep = ',', encoding = 'latin') #separador mudou de ';' para ','
tac()

Time passed: 0hour:4min:11sec


In [6]:
#Renomeando as colunas para manter a consistência
dict_cols = {'COD_MUNICIPIO_RESIDENCIA':'CO_MUNICIPIO_RESIDENCIA',
             'COD_UF_RESIDENCIA':'CO_UF_RESIDENCIA',
             'UF_RESIDENCIA':'SG_UF_RESIDENCIA',
             'IDADE':'NU_IDADE',
             'ST_CONCLUSAO':'TP_ST_CONCLUSAO',
             'IN_TP_ENSINO':'TP_ENSINO',
             'COD_ESCOLA':'CO_ESCOLA',
             'COD_MUNICIPIO_ESC':'CO_MUNICIPIO_ESC',
             'COD_UF_ESC':'CO_UF_ESC',
             'UF_ESC':'SG_UF_ESC',
             'ID_DEPENDENCIA_ADM_ESC':'TP_DEPENDENCIA_ADM_ESC',
             'ID_LOCALIZACAO_ESC':'TP_LOCALIZACAO_ESC',
             'SIT_FUNC_ESC':'TP_SIT_FUNC_ESC',
             'COD_MUNICIPIO_PROVA':'CO_MUNICIPIO_PROVA',
             'COD_UF_PROVA':'CO_UF_PROVA',
             'UF_PROVA':'SG_UF_PROVA',
             'IN_PRESENCA_CN':'TP_PRESENCA_CN',
             'IN_PRESENCA_CH':'TP_PRESENCA_CH',
             'IN_PRESENCA_LC':'TP_PRESENCA_LC',
             'IN_PRESENCA_MT':'TP_PRESENCA_MT',
             'NOTA_CN':'NU_NOTA_CN',
             'NOTA_CH':'NU_NOTA_CH',
             'NOTA_LC':'NU_NOTA_LC',
             'NOTA_MT':'NU_NOTA_MT',
             'IN_STATUS_REDACAO':'TP_STATUS_REDACAO'
             }
new_cols = [dict_cols[c] if c in dict_cols.keys() else c for c in df.columns]
df.columns = new_cols

In [7]:
#Ajustando dados 1: TP_ESTADO_CIVIL
#--------------------------------------------------------------------------------------
#NaN                                       → 0 Não informado 
#0 Solteiro(a)                             → 1 Solteiro(a)
#1 Casado(a)/Mora com companheiro(a)       → 2 Casado(a)/Mora com companheiro(a)
#2 Divorciado(a)/Desquitado(a)/Separado(a) → 3 Divorciado(a)/Desquitado(a)/Separado(a)
#3 Viúvo(a)                                → 4 Viúvo(a)
#--------------------------------------------------------------------------------------
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())
df['TP_ESTADO_CIVIL'] = df['TP_ESTADO_CIVIL'].map({0:1, 1:2, 2:3, 3:4})
df['TP_ESTADO_CIVIL'].fillna(0, inplace = True)
print();print(50*'-');print()
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())

0    7222149
1    1320070
2     162048
3      17981
Name: TP_ESTADO_CIVIL, dtype: int64

--------------------------------------------------

1    7222149
2    1320070
3     162048
4      17981
Name: TP_ESTADO_CIVIL, dtype: int64


In [8]:
#Ajustando dados 2: TP_ENSINO
#--------------------------------------------------------------------------------------
#1 Ensino Regular             → 1 Ensino Regular
#2 Ensino de Jovens e Adultos → 3 Educação de Jovens e Adultos
#4 Ensino Especial            → 2 Educação Especial - Modalidade Substitutiva
#--------------------------------------------------------------------------------------
print(df['TP_ENSINO'].value_counts(dropna = False).sort_index())
df['TP_ENSINO'] = df['TP_ENSINO'].map({1:1, 2:3, 4:2})
print();print(50*'-');print()
print(df['TP_ENSINO'].value_counts(dropna = False).sort_index())

1.0    5958969
2.0     732046
4.0      41841
NaN    1989392
Name: TP_ENSINO, dtype: int64

--------------------------------------------------

1.0    5958969
2.0      41841
3.0     732046
NaN    1989392
Name: TP_ENSINO, dtype: int64


In [9]:
#Ajustando dados 3: TP_STATUS_REDACAO
#--------------------------------------------------------------------------------------
#1 Em Branco                                        → 4 Em Branco
#2 Anulada                                          → 2 Anulada
#3 Fuga ao Tema                                     → 6 Fuga ao tema
#4 Não atende ao tipo textual                       → 7 Não atendimento ao tipo textual
#5 Texto Insuficiente                               → 8 Texto insuficiente
#6 Ausente                                          → 4 Em Branco
#7 Presente e texto conforme os requisitos          → 1 Sem problemas 
#9 Anulada – Fere Direitos Humanos                  → 5 Fere Direitos Humanos
#10 Cópia de texto motivador                        → 3 Cópia Texto Motivador
#11 Parte do texto desconectada com o tema proposto → 9 Parte desconectada
#--------------------------------------------------------------------------------------
print(df['TP_STATUS_REDACAO'].value_counts(dropna = False).sort_index())
df['TP_STATUS_REDACAO'] = df['TP_STATUS_REDACAO'].map({1:4, 2:2, 3:6, 4:7, 5:8, 6:4, 7:1, 9:5, 10:3, 11:9})
print();print(50*'-');print()
print(df['TP_STATUS_REDACAO'].value_counts(dropna = False).sort_index())

1       72538
2        1456
3      216239
4        3614
5        7477
6     2739440
7     5664485
9         939
10      12809
11       3251
Name: TP_STATUS_REDACAO, dtype: int64

--------------------------------------------------

1    5664485
2       1456
3      12809
4    2811978
5        939
6     216239
7       3614
8       7477
9       3251
Name: TP_STATUS_REDACAO, dtype: int64


In [10]:
#Selecionando apenas os campos de interesse
df = df[campos].copy()
print(df.shape)

(8722248, 50)


In [11]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO
0,RESIDENCIA,3548708.0,SAO BERNARDO DO CAMPO,São Bernardo do Campo
1,RESIDENCIA,3118601.0,CONTAGEM,Contagem
2,RESIDENCIA,3550308.0,SAO PAULO,São Paulo
3,RESIDENCIA,2933307.0,VITORIA DA CONQUISTA,Vitória da Conquista
4,RESIDENCIA,3302007.0,ITAGUAI,Itaguaí
...,...,...,...,...
12863,PROVA,1300102.0,ANORI,Anori
12864,PROVA,1304401.0,URUCURITUBA,Urucurituba
12865,PROVA,1300631.0,BERURI,Beruri
12866,PROVA,1302801.0,MARAA,Maraã


In [12]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
for c in ['RESIDENCIA','ESC','PROVA']:
    print(c, sum(~pd.Series(df[~df[f'CO_MUNICIPIO_{c}'].isna()][f'CO_MUNICIPIO_{c}'].unique()).isin(dict_municipios.keys()))==0,
          sep = ':    \t')

RESIDENCIA:    	True
ESC:    	True
PROVA:    	True


In [13]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
for c in ['RESIDENCIA','ESC','PROVA']: df[f'NO_MUNICIPIO_{c}'] = df[f'CO_MUNICIPIO_{c}'].map(dict_municipios)

In [14]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO


In [15]:
#Ajustando dados: SG_UF_RESIDENCIA/ESC/PROVA
check_ufs(df)

,COL,CO_MUNICIPIO,SG_MUNICIPIO,SIGLA_ESPERADA


In [16]:
#Verificando a consistência dos dados e sumarizando estatísticas descritivas
year_res_dict = {year:{'cols':df.shape[1], 'rows':df.shape[0], 'data':{}}}
tic()
for col in campos:
    res_prt, res_val = validate(df, col, year, get_values=True)
    print_results(col, res_prt); print()
    year_res_dict[year]['data'][col] = res_val
tac()

NU_INSCRICAO
-------------------------
Tipo:     	True
Tamanho:  	True
Unicidade:	True
Nulos:    	0

NU_ANO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

CO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

CO_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

SG_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NU_IDADE
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	227

TP_SEXO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ESTADO_CIVIL
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_COR_RACA
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ST_CONCLUSAO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_

In [17]:
#Armazenando as estatísticas descritivas sumarizadas em um arquivo json
with open(save_dir(year)['json'], 'w', encoding='utf-8') as fp: json.dump(year_res_dict, fp, cls=NpEncoder, ensure_ascii=False)

In [18]:
#Armazenando os dados do ano em um arquivo csv
tic()
df.to_csv(save_dir(year)['csv'], sep=';', header = True, index = False, encoding='latin')
tac()

Time passed: 0hour:1min:55sec


___

## 2013

In [23]:
#Obtendo informações do ano
year = 2013
tic()
df = pd.read_csv(get_dir(year), sep = ';', encoding = 'latin')
tac()

Time passed: 0hour:3min:52sec


In [26]:
#Renomeando as colunas para manter a consistência
dict_cols = {'COD_MUNICIPIO_RESIDENCIA':'CO_MUNICIPIO_RESIDENCIA',
             'COD_UF_RESIDENCIA':'CO_UF_RESIDENCIA',
             'UF_RESIDENCIA':'SG_UF_RESIDENCIA',
             'IDADE':'NU_IDADE',
             'ST_CONCLUSAO':'TP_ST_CONCLUSAO',
             'IN_TP_ENSINO':'TP_ENSINO',
             'COD_ESCOLA':'CO_ESCOLA',
             'COD_MUNICIPIO_ESC':'CO_MUNICIPIO_ESC',
             'COD_UF_ESC':'CO_UF_ESC',
             'UF_ESC':'SG_UF_ESC',
             'ID_DEPENDENCIA_ADM_ESC':'TP_DEPENDENCIA_ADM_ESC',
             'ID_LOCALIZACAO_ESC':'TP_LOCALIZACAO_ESC',
             'SIT_FUNC_ESC':'TP_SIT_FUNC_ESC',
             'COD_MUNICIPIO_PROVA':'CO_MUNICIPIO_PROVA',
             'COD_UF_PROVA':'CO_UF_PROVA',
             'UF_PROVA':'SG_UF_PROVA',
             'IN_PRESENCA_CN':'TP_PRESENCA_CN',
             'IN_PRESENCA_CH':'TP_PRESENCA_CH',
             'IN_PRESENCA_LC':'TP_PRESENCA_LC',
             'IN_PRESENCA_MT':'TP_PRESENCA_MT',
             'NOTA_CN':'NU_NOTA_CN',
             'NOTA_CH':'NU_NOTA_CH',
             'NOTA_LC':'NU_NOTA_LC',
             'NOTA_MT':'NU_NOTA_MT',
             'IN_STATUS_REDACAO':'TP_STATUS_REDACAO'
             }
new_cols = [dict_cols[c] if c in dict_cols.keys() else c for c in df.columns]
df.columns = new_cols

In [32]:
#Ajustando dados 1: TP_ESTADO_CIVIL
#--------------------------------------------------------------------------------------
#NaN                                       → 0 Não informado 
#0 Solteiro(a)                             → 1 Solteiro(a)
#1 Casado(a)/Mora com companheiro(a)       → 2 Casado(a)/Mora com companheiro(a)
#2 Divorciado(a)/Desquitado(a)/Separado(a) → 3 Divorciado(a)/Desquitado(a)/Separado(a)
#3 Viúvo(a)                                → 4 Viúvo(a)
#--------------------------------------------------------------------------------------
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())
df['TP_ESTADO_CIVIL'] = df['TP_ESTADO_CIVIL'].map({0:1, 1:2, 2:3, 3:4})
df['TP_ESTADO_CIVIL'].fillna(0, inplace = True)
print();print(50*'-');print()
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())

0    5970943
1    1057371
2     130561
3      14688
Name: TP_ESTADO_CIVIL, dtype: int64

--------------------------------------------------

1    5970943
2    1057371
3     130561
4      14688
Name: TP_ESTADO_CIVIL, dtype: int64


In [33]:
#Ajustando dados 2: TP_ENSINO
#--------------------------------------------------------------------------------------
#1 Ensino Regular             → 1 Ensino Regular
#2 Ensino de Jovens e Adultos → 3 Educação de Jovens e Adultos
#4 Ensino Especial            → 2 Educação Especial - Modalidade Substitutiva
#--------------------------------------------------------------------------------------
print(df['TP_ENSINO'].value_counts(dropna = False).sort_index())
df['TP_ENSINO'] = df['TP_ENSINO'].map({1:1, 2:3, 4:2})
print();print(50*'-');print()
print(df['TP_ENSINO'].value_counts(dropna = False).sort_index())

1.0    5040277
2.0     589358
4.0      40675
NaN    1503253
Name: TP_ENSINO, dtype: int64

--------------------------------------------------

1.0    5040277
2.0      40675
3.0     589358
NaN    1503253
Name: TP_ENSINO, dtype: int64


In [34]:
#Ajustando dados 3: TP_STATUS_REDACAO
#--------------------------------------------------------------------------------------
#1 Em Branco                                        → 4 Em Branco
#2 Anulada                                          → 2 Anulada
#3 Fuga ao Tema                                     → 6 Fuga ao tema
#4 Não atende ao tipo textual                       → 7 Não atendimento ao tipo textual
#5 Texto Insuficiente                               → 8 Texto insuficiente
#6 Ausente                                          → 4 Em Branco
#7 Presente e texto conforme os requisitos          → 1 Sem problemas 
#9 Anulada – Fere Direitos Humanos                  → 5 Fere Direitos Humanos
#10 Cópia de texto motivador                        → 3 Cópia Texto Motivador
#11 Parte do texto desconectada com o tema proposto → 9 Parte desconectada
#--------------------------------------------------------------------------------------
print(df['TP_STATUS_REDACAO'].value_counts(dropna = False).sort_index())
df['TP_STATUS_REDACAO'] = df['TP_STATUS_REDACAO'].map({1:4, 2:2, 3:6, 4:7, 5:8, 6:4, 7:1, 9:5, 10:3, 11:9})
print();print(50*'-');print()
print(df['TP_STATUS_REDACAO'].value_counts(dropna = False).sort_index())

1       33307
2        1171
3       52321
4        6775
5        4521
6     2140794
7     4929862
9         164
10       3250
11       1398
Name: TP_STATUS_REDACAO, dtype: int64

--------------------------------------------------

1    4929862
2       1171
3       3250
4    2174101
5        164
6      52321
7       6775
8       4521
9       1398
Name: TP_STATUS_REDACAO, dtype: int64


In [35]:
#Selecionando apenas os campos de interesse
df = df[campos].copy()
print(df.shape)

(7173563, 50)


In [36]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO
0,RESIDENCIA,4314902.0,PORTO ALEGRE,Porto Alegre
1,RESIDENCIA,1501402.0,BELEM,Belém
2,RESIDENCIA,2902500.0,BAIANOPOLIS,Baianópolis
3,RESIDENCIA,3544004.0,RIO DAS PEDRAS,Rio das Pedras
4,RESIDENCIA,2402006.0,CAICO,Caicó
...,...,...,...,...
12763,PROVA,1200252.0,EPITACIOLANDIA,Epitaciolândia
12764,PROVA,1200385.0,PLACIDO DE CASTRO,Plácido de Castro
12765,PROVA,1200351.0,MARECHAL THAUMATURGO,Marechal Thaumaturgo
12766,PROVA,1200393.0,PORTO WALTER,Porto Walter


In [37]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
for c in ['RESIDENCIA','ESC','PROVA']:
    print(c, sum(~pd.Series(df[~df[f'CO_MUNICIPIO_{c}'].isna()][f'CO_MUNICIPIO_{c}'].unique()).isin(dict_municipios.keys()))==0,
          sep = ':    \t')

RESIDENCIA:    	True
ESC:    	True
PROVA:    	True


In [38]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
for c in ['RESIDENCIA','ESC','PROVA']: df[f'NO_MUNICIPIO_{c}'] = df[f'CO_MUNICIPIO_{c}'].map(dict_municipios)

In [39]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO


In [40]:
#Ajustando dados: SG_UF_RESIDENCIA/ESC/PROVA
check_ufs(df)

,COL,CO_MUNICIPIO,SG_MUNICIPIO,SIGLA_ESPERADA


In [41]:
#Verificando a consistência dos dados e sumarizando estatísticas descritivas
year_res_dict = {year:{'cols':df.shape[1], 'rows':df.shape[0], 'data':{}}}
tic()
for col in campos:
    res_prt, res_val = validate(df, col, year, get_values=True)
    print_results(col, res_prt); print()
    year_res_dict[year]['data'][col] = res_val
tac()

NU_INSCRICAO
-------------------------
Tipo:     	True
Tamanho:  	True
Unicidade:	True
Nulos:    	0

NU_ANO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

CO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

CO_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

SG_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NU_IDADE
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	243

TP_SEXO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ESTADO_CIVIL
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_COR_RACA
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ST_CONCLUSAO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_

In [42]:
#Armazenando as estatísticas descritivas sumarizadas em um arquivo json
with open(save_dir(year)['json'], 'w', encoding='utf-8') as fp: json.dump(year_res_dict, fp, cls=NpEncoder, ensure_ascii=False)

In [43]:
#Armazenando os dados do ano em um arquivo csv
tic()
df.to_csv(save_dir(year)['csv'], sep=';', header = True, index = False, encoding='latin')
tac()

Time passed: 0hour:2min:12sec


___

## 2012

In [5]:
#Obtendo informações do ano
year = 2012
tic()
df = pd.read_csv(get_dir(year), sep = ',', encoding = 'latin')
tac()

Time passed: 0hour:0min:40sec


In [6]:
#Recriando os codigos das UFs
df['CO_UF_RESIDENCIA'] = df['UF_INSC'].map(dict_ufs_inv)
print('CO_UF_RESIDENCIA', df['CO_UF_RESIDENCIA'].isna().sum() == df['UF_INSC'].isna().sum(), sep = ': ')
df['CO_UF_ESC'] = df['UF_ESC'].map(dict_ufs_inv)
print('CO_UF_ESC', df['CO_UF_ESC'].isna().sum() == df['UF_ESC'].isna().sum(), sep = ': ')
df['CO_UF_PROVA'] = df['UF_MUNICIPIO_PROVA'].map(dict_ufs_inv)
print('CO_UF_PROVA', df['CO_UF_PROVA'].isna().sum() == df['UF_MUNICIPIO_PROVA'].isna().sum(), sep = ': ')

CO_UF_RESIDENCIA: True
CO_UF_ESC: True
CO_UF_PROVA: True


In [7]:
#Renomeando as colunas para manter a consistência
dict_cols = {'COD_MUNICIPIO_INSC':'CO_MUNICIPIO_RESIDENCIA',
             'NO_MUNICIPIO_INSC':'NO_MUNICIPIO_RESIDENCIA',
             #'COD_UF_RESIDENCIA':'CO_UF_RESIDENCIA', -- GERADO A PARTIR DO CAMPO 'SG_UF_RESIDENCIA'
             'UF_INSC':'SG_UF_RESIDENCIA',
             'IDADE':'NU_IDADE',
             'ST_CONCLUSAO':'TP_ST_CONCLUSAO',
             'IN_TP_ENSINO':'TP_ENSINO',
             'PK_COD_ENTIDADE':'CO_ESCOLA',
             'COD_MUNICIPIO_ESC':'CO_MUNICIPIO_ESC',
             #'COD_UF_ESC':'CO_UF_ESC', -- GERADO A PARTIR DO CAMPO 'SG_UF_ESC'
             'UF_ESC':'SG_UF_ESC',
             'ID_DEPENDENCIA_ADM':'TP_DEPENDENCIA_ADM_ESC',
             'ID_LOCALIZACAO':'TP_LOCALIZACAO_ESC',
             'SIT_FUNC':'TP_SIT_FUNC_ESC',
             'COD_MUNICIPIO_PROVA':'CO_MUNICIPIO_PROVA',
             #'COD_UF_PROVA':'CO_UF_PROVA', -- GERADO A PARTIR DO CAMPO 'SG_UF_PROVA'
             'UF_MUNICIPIO_PROVA':'SG_UF_PROVA',
             'IN_PRESENCA_CN':'TP_PRESENCA_CN',
             'IN_PRESENCA_CH':'TP_PRESENCA_CH',
             'IN_PRESENCA_LC':'TP_PRESENCA_LC',
             'IN_PRESENCA_MT':'TP_PRESENCA_MT',
             'NU_NT_CN':'NU_NOTA_CN',
             'NU_NT_CH':'NU_NOTA_CH',
             'NU_NT_LC':'NU_NOTA_LC',
             'NU_NT_MT':'NU_NOTA_MT',
             'IN_STATUS_REDACAO':'TP_STATUS_REDACAO'
             }
new_cols = [dict_cols[c] if c in dict_cols.keys() else c for c in df.columns]
df.columns = new_cols

In [8]:
#Ajustando dados 1: substituindo '.' por None
num_campos = ['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'NU_IDADE', 'TP_ESTADO_CIVIL',
              'TP_COR_RACA', 'TP_ST_CONCLUSAO', 'TP_ENSINO', 'CO_ESCOLA', 'CO_MUNICIPIO_ESC', 'CO_UF_ESC',
              'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA',
              'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA', 'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL', 'IN_DEFICIT_ATENCAO',
              'IN_DISLEXIA', 'IN_GESTANTE', 'IN_LACTANTE', 'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'TP_PRESENCA_CN',
              'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
              'TP_LINGUA', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
              'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'] 

for c in df.columns: 
    df.loc[df[c]=='.', c] = None
    if c in num_campos:
        df[c] = pd.to_numeric(df[c])

In [9]:
#Ajustando dados 2: TP_SEXO
#--------------------------------------------------------------------------------------
#0 → M
#1 → F
#--------------------------------------------------------------------------------------
print(df['TP_SEXO'].value_counts(dropna = False).sort_index())
df['TP_SEXO'] = df['TP_SEXO'].map({0:'M', 1:'F'})
print();print(50*'-');print()
print(df['TP_SEXO'].value_counts(dropna = False).sort_index())

0.0    2374655
1.0    3416410
Name: TP_SEXO, dtype: int64

--------------------------------------------------

F    3416410
M    2374655
Name: TP_SEXO, dtype: int64


In [10]:
#Ajustando dados 3: TP_ESTADO_CIVIL
#--------------------------------------------------------------------------------------
#NaN                                       → 0 Não informado 
#0 Solteiro(a)                             → 1 Solteiro(a)
#1 Casado(a)/Mora com companheiro(a)       → 2 Casado(a)/Mora com companheiro(a)
#2 Divorciado(a)/Desquitado(a)/Separado(a) → 3 Divorciado(a)/Desquitado(a)/Separado(a)
#3 Viúvo(a)                                → 4 Viúvo(a)
#--------------------------------------------------------------------------------------
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())
df['TP_ESTADO_CIVIL'] = df['TP_ESTADO_CIVIL'].map({0:1, 1:2, 2:3, 3:4})
df['TP_ESTADO_CIVIL'].fillna(0, inplace = True)
print();print(50*'-');print()
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())

0.0    4820621
1.0     853915
2.0     103345
3.0      13184
Name: TP_ESTADO_CIVIL, dtype: int64

--------------------------------------------------

1    4820621
2     853915
3     103345
4      13184
Name: TP_ESTADO_CIVIL, dtype: int64


In [11]:
#Ajustando dados 4: TP_ENSINO
#--------------------------------------------------------------------------------------
#1 Ensino Regular             → 1 Ensino Regular
#2 Ensino de Jovens e Adultos → 3 Educação de Jovens e Adultos
#3 Ensino Profissionalizante  → 4 Ensino Profissionalizante
#4 Ensino Especial            → 2 Educação Especial - Modalidade Substitutiva
#--------------------------------------------------------------------------------------
print(df['TP_ENSINO'].value_counts(dropna = False).sort_index())
df['TP_ENSINO'] = df['TP_ENSINO'].map({1:1, 2:3, 3:4, 4:2})
print();print(50*'-');print()
print(df['TP_ENSINO'].value_counts(dropna = False).sort_index())

1.0    4252189
2.0     473422
4.0      34811
NaN    1030643
Name: TP_ENSINO, dtype: int64

--------------------------------------------------

1.0    4252189
2.0      34811
3.0     473422
NaN    1030643
Name: TP_ENSINO, dtype: int64


In [12]:
#Ajustando dados 5: TP_STATUS_REDACAO
#--------------------------------------------------------------------------------------
#P Presente                        → 1 Sem problemas
#B Em Branco                       → 4 Em Branco
#T Fuga ao tema                    → 6 Fuga ao tema
#N Anulada                         → 2 Anulada
#I Texto insuficiente              → 8 Texto insuficiente
#A Não atendimento ao tipo textual → 7 Não atendimento ao tipo textual
#H Anulada – Fere Direitos Humanos → 5 Fere Direitos Humanos
#C Cópia de texto motivador        → 3 Cópia Texto Motivador
#F Ausente                         → 4 Em Branco
#--------------------------------------------------------------------------------------
print(df['TP_STATUS_REDACAO'].value_counts(dropna = False).sort_index())
df['TP_STATUS_REDACAO'] = pd.to_numeric(df['TP_STATUS_REDACAO'].map({'P':1, 'B':4, 'T':6, 'N':2, 'I':8, 'A':7, 'H':5, 'C':3, 'F':4}))
print();print(50*'-');print()
print(df['TP_STATUS_REDACAO'].value_counts(dropna = False).sort_index())

A       1791
B      72818
C       7733
F    1694441
H         79
I       5328
N       1274
P    3955218
T      52383
Name: TP_STATUS_REDACAO, dtype: int64

--------------------------------------------------

1    3955218
2       1274
3       7733
4    1767259
5         79
6      52383
7       1791
8       5328
Name: TP_STATUS_REDACAO, dtype: int64


In [13]:
#Ajustando dados 6: NU_IDADE
df.loc[~(df['NU_IDADE'].isin(range(10, 101))), 'NU_IDADE'] = None

In [14]:
#Selecionando apenas os campos de interesse
df = df[campos].copy()
print(df.shape)

(5791065, 50)


In [15]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO
0,RESIDENCIA,2933307.0,VITORIA DA CONQUISTA,Vitória da Conquista
1,RESIDENCIA,2611606.0,RECIFE,Recife
2,RESIDENCIA,3301702.0,DUQUE DE CAXIAS,Duque de Caxias
3,RESIDENCIA,5300108.0,BRASILIA,Brasília
4,RESIDENCIA,3304557.0,RIO DE JANEIRO,Rio de Janeiro
...,...,...,...,...
12711,PROVA,1703701.0,BREJINHO DE NAZARE,Brejinho de Nazaré
12712,PROVA,1200393.0,PORTO WALTER,Porto Walter
12713,PROVA,1301159.0,CAREIRO DA VARZEA,Careiro da Várzea
12714,PROVA,1707405.0,ESPERANTINA,Esperantina


In [16]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
for c in ['RESIDENCIA','ESC','PROVA']:
    print(c, sum(~pd.Series(df[~df[f'CO_MUNICIPIO_{c}'].isna()][f'CO_MUNICIPIO_{c}'].unique()).isin(dict_municipios.keys()))==0,
          sep = ':    \t')

RESIDENCIA:    	True
ESC:    	True
PROVA:    	True


In [17]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
for c in ['RESIDENCIA','ESC','PROVA']: df[f'NO_MUNICIPIO_{c}'] = df[f'CO_MUNICIPIO_{c}'].map(dict_municipios)

In [18]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO


In [19]:
#Ajustando dados: SG_UF_RESIDENCIA/ESC/PROVA
check_ufs(df)

,COL,CO_MUNICIPIO,SG_MUNICIPIO,SIGLA_ESPERADA


In [20]:
#Verificando a consistência dos dados e sumarizando estatísticas descritivas
year_res_dict = {year:{'cols':df.shape[1], 'rows':df.shape[0], 'data':{}}}
tic()
for col in campos:
    res_prt, res_val = validate(df, col, year, get_values=True)
    print_results(col, res_prt); print()
    year_res_dict[year]['data'][col] = res_val
tac()

NU_INSCRICAO
-------------------------
Tipo:     	True
Tamanho:  	True
Unicidade:	True
Nulos:    	0

NU_ANO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

CO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

CO_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

SG_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NU_IDADE
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	160

TP_SEXO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ESTADO_CIVIL
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_COR_RACA
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ST_CONCLUSAO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_

In [21]:
#Armazenando as estatísticas descritivas sumarizadas em um arquivo json
with open(save_dir(year)['json'], 'w', encoding='utf-8') as fp: json.dump(year_res_dict, fp, cls=NpEncoder, ensure_ascii=False)

In [22]:
#Armazenando os dados do ano em um arquivo csv
tic()
df.to_csv(save_dir(year)['csv'], sep=';', header = True, index = False, encoding='latin')
tac()

Time passed: 0hour:2min:36sec


___

## 2011

In [35]:
#Alterando formato dos dados em 2011
def get_data_txt(txt):
    val_dict = {
    'NU_INSCRICAO': txt[0:(11+1)].strip(), 
    'NU_ANO': txt[12:(15+1)].strip(), 
    'IDADE': txt[16:(18+1)].strip(), 
    'TP_SEXO': txt[19:(19+1)].strip(), 
    'COD_MUNICIPIO_INSC': txt[20:(26+1)].strip(), 
    'NO_MUNICIPIO_INSC': txt[27:(176+1)].strip(), 
    'UF_INSC': txt[177:(178+1)].strip(), 
    'ST_CONCLUSAO': txt[179:(179+1)].strip(), 
    'IN_TP_ENSINO': txt[180:(180+1)].strip(), 
    'IN_CERTIFICADO': txt[181:(181+1)].strip(), 
    'IN_BRAILLE': txt[182:(182+1)].strip(), 
    'IN_AMPLIADA': txt[183:(183+1)].strip(), 
    'IN_LEDOR': txt[184:(184+1)].strip(), 
    'IN_ACESSO': txt[185:(185+1)].strip(), 
    'IN_TRANSCRICAO': txt[186:(186+1)].strip(), 
    'IN_LIBRAS': txt[187:(187+1)].strip(), 
    'IN_UNIDADE_PRISIONAL': txt[188:(188+1)].strip(), 
    'IN_BAIXA_VISAO': txt[189:(189+1)].strip(), 
    'IN_CEGUEIRA': txt[190:(190+1)].strip(), 
    'IN_DEFICIENCIA_AUDITIVA': txt[191:(191+1)].strip(), 
    'IN_DEFICIENCIA_FISICA': txt[192:(192+1)].strip(), 
    'IN_DEFICIENCIA_MENTAL': txt[193:(193+1)].strip(), 
    'IN_DEFICIT_ATENCAO': txt[194:(194+1)].strip(), 
    'IN_DISLEXIA': txt[195:(195+1)].strip(), 
    'IN_GESTANTE': txt[196:(196+1)].strip(), 
    'IN_LACTANTE': txt[197:(197+1)].strip(), 
    'IN_LEITURA_LABIAL': txt[198:(198+1)].strip(), 
    'IN_SABATISTA': txt[199:(199+1)].strip(), 
    'IN_SURDEZ': txt[200:(200+1)].strip(), 
    'TP_ESTADO_CIVIL': txt[201:(201+1)].strip(), 
    'TP_COR_RACA': txt[202:(202+1)].strip(), 
    'PK_COD_ENTIDADE': txt[203:(210+1)].strip(), 
    'COD_MUNICIPIO_ESC': txt[211:(217+1)].strip(), 
    'NO_MUNICIPIO_ESC': txt[218:(367+1)].strip(), 
    'UF_ESC': txt[368:(369+1)].strip(), 
    'ID_DEPENDENCIA_ADM': txt[370:(370+1)].strip(), 
    'ID_LOCALIZACAO': txt[371:(371+1)].strip(), 
    'SIT_FUNC': txt[372:(372+1)].strip(), 
    'COD_MUNICIPIO_PROVA': txt[373:(379+1)].strip(), 
    'NO_MUNICIPIO_PROVA': txt[380:(529+1)].strip(), 
    'UF_MUNICIPIO_PROVA': txt[530:(531+1)].strip(), 
    'IN_PRESENCA_CN': txt[532:(532+1)].strip(), 
    'IN_PRESENCA_CH': txt[533:(533+1)].strip(), 
    'IN_PRESENCA_LC': txt[534:(534+1)].strip(), 
    'IN_PRESENCA_MT': txt[535:(535+1)].strip(), 
    'NU_NT_CN': txt[536:(544+1)].strip(), 
    'NU_NT_CH': txt[545:(553+1)].strip(), 
    'NU_NT_LC': txt[554:(562+1)].strip(), 
    'NU_NT_MT': txt[563:(571+1)].strip(), 
    'TX_RESPOSTAS_CN': txt[572:(616+1)].strip(), 
    'TX_RESPOSTAS_CH': txt[617:(661+1)].strip(), 
    'TX_RESPOSTAS_LC': txt[662:(706+1)].strip(), 
    'TX_RESPOSTAS_MT': txt[707:(751+1)].strip(), 
    'ID_PROVA_CN': txt[752:(754+1)].strip(), 
    'ID_PROVA_CH': txt[755:(757+1)].strip(), 
    'ID_PROVA_LC': txt[758:(760+1)].strip(), 
    'ID_PROVA_MT': txt[761:(763+1)].strip(), 
    'TP_LINGUA': txt[764:(764+1)].strip(), 
    'DS_GABARITO_CN': txt[765:(809+1)].strip(), 
    'DS_GABARITO_CH': txt[810:(854+1)].strip(), 
    'DS_GABARITO_LC': txt[855:(904+1)].strip(), 
    'DS_GABARITO_MT': txt[905:(949+1)].strip(), 
    'IN_STATUS_REDACAO': txt[950:(950+1)].strip(), 
    'NU_NOTA_COMP1': txt[951:(959+1)].strip(), 
    'NU_NOTA_COMP2': txt[960:(968+1)].strip(), 
    'NU_NOTA_COMP3': txt[969:(977+1)].strip(), 
    'NU_NOTA_COMP4': txt[978:(986+1)].strip(), 
    'NU_NOTA_COMP5': txt[987:(995+1)].strip(), 
    'NU_NOTA_REDACAO': txt[996:(1004+1)].strip(), 
    'IN_CONCLUINTE_CENSO': txt[1005:(1005+1)].strip(), 
    'COD_ETAPA_ENSINO_CENSO': txt[1006:(1007+1)].strip(), 
    'COD_ENTIDADE_CENSO': txt[1008:(1015+1)].strip(), 
    'COD_MUNICIPIO_ESC_CENSO': txt[1016:(1022+1)].strip(), 
    'NO_MUNICIPIO_ESC_CENSO': txt[1023:(1172+1)].strip(), 
    'UF_ESC_CENSO': txt[1173:(1174+1)].strip(), 
    'ID_DEPENDENCIA_ADM_CENSO': txt[1175:(1175+1)].strip(), 
    'ID_LOCALIZACAO_CENSO': txt[1176:(1176+1)].strip(), 
    'SIT_FUNC_CENSO': txt[1177:(1177+1)].strip()    
}
    return val_dict

cols = ['NU_INSCRICAO', 'NU_ANO', 'IDADE', 'TP_SEXO', 'COD_MUNICIPIO_INSC', 'NO_MUNICIPIO_INSC', 'UF_INSC', 'ST_CONCLUSAO',
        'IN_TP_ENSINO', 'IN_CERTIFICADO', 'IN_BRAILLE', 'IN_AMPLIADA', 'IN_LEDOR', 'IN_ACESSO', 'IN_TRANSCRICAO', 'IN_LIBRAS',
        'IN_UNIDADE_PRISIONAL', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_DEFICIENCIA_AUDITIVA', 'IN_DEFICIENCIA_FISICA',
        'IN_DEFICIENCIA_MENTAL', 'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_GESTANTE', 'IN_LACTANTE', 'IN_LEITURA_LABIAL',
        'IN_SABATISTA', 'IN_SURDEZ', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'PK_COD_ENTIDADE', 'COD_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC',
        'UF_ESC', 'ID_DEPENDENCIA_ADM', 'ID_LOCALIZACAO', 'SIT_FUNC', 'COD_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA',
        'UF_MUNICIPIO_PROVA', 'IN_PRESENCA_CN', 'IN_PRESENCA_CH', 'IN_PRESENCA_LC', 'IN_PRESENCA_MT', 'NU_NT_CN', 'NU_NT_CH',
        'NU_NT_LC', 'NU_NT_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'ID_PROVA_CN',
        'ID_PROVA_CH', 'ID_PROVA_LC', 'ID_PROVA_MT', 'TP_LINGUA', 'DS_GABARITO_CN', 'DS_GABARITO_CH', 'DS_GABARITO_LC',
        'DS_GABARITO_MT', 'IN_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
        'NU_NOTA_REDACAO', 'IN_CONCLUINTE_CENSO', 'COD_ETAPA_ENSINO_CENSO', 'COD_ENTIDADE_CENSO', 'COD_MUNICIPIO_ESC_CENSO',
        'NO_MUNICIPIO_ESC_CENSO', 'UF_ESC_CENSO', 'ID_DEPENDENCIA_ADM_CENSO', 'ID_LOCALIZACAO_CENSO', 'SIT_FUNC_CENSO']

file = os.getcwd()+'\\ENEM\\2011\\DADOS\\DADOS_ENEM_2011.TXT'
new_file = os.getcwd()+'\\ENEM\\2011\\DADOS\\MICRODADOS_ENEM_2011.csv'

count = 0
with open(new_file, 'a') as nfl:
    nfl.write(';'.join(cols)+'\n')
    with open(file) as fl:
        for line in fl:
            nfl.write(';'.join(get_data_txt(line).values())+'\n')
            count += 1

In [5]:
#Obtendo informações do ano
year = 2011
tic()
df = pd.read_csv(get_dir(year), sep = ';', encoding = 'latin', dtype = 'O')
tac()

Time passed: 0hour:0min:46sec


In [6]:
#Recriando os codigos das UFs
df['CO_UF_RESIDENCIA'] = df['UF_INSC'].map(dict_ufs_inv)
print('CO_UF_RESIDENCIA', df['CO_UF_RESIDENCIA'].isna().sum() == df['UF_INSC'].isna().sum(), sep = ': ')
df['CO_UF_ESC'] = df['UF_ESC'].map(dict_ufs_inv)
print('CO_UF_ESC', df['CO_UF_ESC'].isna().sum() == df['UF_ESC'].isna().sum(), sep = ': ')
df['CO_UF_PROVA'] = df['UF_MUNICIPIO_PROVA'].map(dict_ufs_inv)
print('CO_UF_PROVA', df['CO_UF_PROVA'].isna().sum() == df['UF_MUNICIPIO_PROVA'].isna().sum(), sep = ': ')

CO_UF_RESIDENCIA: True
CO_UF_ESC: True
CO_UF_PROVA: True


In [7]:
#Renomeando as colunas para manter a consistência
dict_cols = {'COD_MUNICIPIO_INSC':'CO_MUNICIPIO_RESIDENCIA',
             'NO_MUNICIPIO_INSC':'NO_MUNICIPIO_RESIDENCIA',
             #'COD_UF_RESIDENCIA':'CO_UF_RESIDENCIA', -- GERADO A PARTIR DO CAMPO 'SG_UF_RESIDENCIA'
             'UF_INSC':'SG_UF_RESIDENCIA',
             'IDADE':'NU_IDADE',
             'ST_CONCLUSAO':'TP_ST_CONCLUSAO',
             'IN_TP_ENSINO':'TP_ENSINO',
             'PK_COD_ENTIDADE':'CO_ESCOLA',
             'COD_MUNICIPIO_ESC':'CO_MUNICIPIO_ESC',
             #'COD_UF_ESC':'CO_UF_ESC', -- GERADO A PARTIR DO CAMPO 'SG_UF_ESC'
             'UF_ESC':'SG_UF_ESC',
             'ID_DEPENDENCIA_ADM':'TP_DEPENDENCIA_ADM_ESC',
             'ID_LOCALIZACAO':'TP_LOCALIZACAO_ESC',
             'SIT_FUNC':'TP_SIT_FUNC_ESC',
             'COD_MUNICIPIO_PROVA':'CO_MUNICIPIO_PROVA',
             #'COD_UF_PROVA':'CO_UF_PROVA', -- GERADO A PARTIR DO CAMPO 'SG_UF_PROVA'
             'UF_MUNICIPIO_PROVA':'SG_UF_PROVA',
             'IN_PRESENCA_CN':'TP_PRESENCA_CN',
             'IN_PRESENCA_CH':'TP_PRESENCA_CH',
             'IN_PRESENCA_LC':'TP_PRESENCA_LC',
             'IN_PRESENCA_MT':'TP_PRESENCA_MT',
             'NU_NT_CN':'NU_NOTA_CN',
             'NU_NT_CH':'NU_NOTA_CH',
             'NU_NT_LC':'NU_NOTA_LC',
             'NU_NT_MT':'NU_NOTA_MT',
             'IN_STATUS_REDACAO':'TP_STATUS_REDACAO'
             }
new_cols = [dict_cols[c] if c in dict_cols.keys() else c for c in df.columns]
df.columns = new_cols

In [8]:
#Ajustando dados 1: substituindo '.' por None
num_campos = ['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'NU_IDADE', 'TP_ESTADO_CIVIL', 'TP_SEXO',
              'TP_COR_RACA', 'TP_ST_CONCLUSAO', 'TP_ENSINO', 'CO_ESCOLA', 'CO_MUNICIPIO_ESC', 'CO_UF_ESC',
              'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA',
              'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA', 'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL', 'IN_DEFICIT_ATENCAO',
              'IN_DISLEXIA', 'IN_GESTANTE', 'IN_LACTANTE', 'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'TP_PRESENCA_CN',
              'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
              'TP_LINGUA', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
              'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'] 

for c in df.columns: 
    df.loc[df[c]=='.', c] = None
    if c in num_campos:
        df[c] = pd.to_numeric(df[c])

In [9]:
#Ajustando dados 2: TP_SEXO
#--------------------------------------------------------------------------------------
#0 → M
#1 → F
#--------------------------------------------------------------------------------------
print(df['TP_SEXO'].value_counts(dropna = False).sort_index())
df['TP_SEXO'] = df['TP_SEXO'].map({0:'M', 1:'F'})
print();print(50*'-');print()
print(df['TP_SEXO'].value_counts(dropna = False).sort_index())

0    2191902
1    3188954
Name: TP_SEXO, dtype: int64

--------------------------------------------------

F    3188954
M    2191902
Name: TP_SEXO, dtype: int64


In [10]:
#Ajustando dados 3: TP_ESTADO_CIVIL
#--------------------------------------------------------------------------------------
#NaN                                       → 0 Não informado 
#0 Solteiro(a)                             → 1 Solteiro(a)
#1 Casado(a)/Mora com companheiro(a)       → 2 Casado(a)/Mora com companheiro(a)
#2 Divorciado(a)/Desquitado(a)/Separado(a) → 3 Divorciado(a)/Desquitado(a)/Separado(a)
#3 Viúvo(a)                                → 4 Viúvo(a)
#--------------------------------------------------------------------------------------
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())
df['TP_ESTADO_CIVIL'] = df['TP_ESTADO_CIVIL'].map({0:1, 1:2, 2:3, 3:4})
df['TP_ESTADO_CIVIL'].fillna(0, inplace = True)
print();print(50*'-');print()
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())

0.0    4424585
1.0     828533
2.0      99881
3.0      13949
NaN      13908
Name: TP_ESTADO_CIVIL, dtype: int64

--------------------------------------------------

0.0      13908
1.0    4424585
2.0     828533
3.0      99881
4.0      13949
Name: TP_ESTADO_CIVIL, dtype: int64


In [11]:
#Ajustando dados 4: TP_ENSINO
#--------------------------------------------------------------------------------------
#1 Ensino Regular             → 1 Ensino Regular
#2 Ensino de Jovens e Adultos → 3 Educação de Jovens e Adultos
#3 Ensino Profissionalizante  → 4 Ensino Profissionalizante
#4 Ensino Especial            → 2 Educação Especial - Modalidade Substitutiva
#--------------------------------------------------------------------------------------
print(df['TP_ENSINO'].value_counts(dropna = False).sort_index())
df['TP_ENSINO'] = df['TP_ENSINO'].map({1:1, 2:3, 3:4, 4:2})
print();print(50*'-');print()
print(df['TP_ENSINO'].value_counts(dropna = False).sort_index())

1.0    4070948
2.0     467448
3.0         35
4.0      36024
NaN     806401
Name: TP_ENSINO, dtype: int64

--------------------------------------------------

1.0    4070948
2.0      36024
3.0     467448
4.0         35
NaN     806401
Name: TP_ENSINO, dtype: int64


In [12]:
#Ajustando dados 5: TP_STATUS_REDACAO
#--------------------------------------------------------------------------------------
#B Em Branco                       → 4 Em Branco
#F Faltoso                         → 4 Em Branco
#N Anulada                         → 2 Anulada
#P Presente                        → 1 Sem problemas
#--------------------------------------------------------------------------------------
print(df['TP_STATUS_REDACAO'].value_counts(dropna = False).sort_index())
df['TP_STATUS_REDACAO'] = pd.to_numeric(df['TP_STATUS_REDACAO'].map({'B':4, 'F':4, 'N':2, 'P':1}))
print();print(50*'-');print()
print(df['TP_STATUS_REDACAO'].value_counts(dropna = False).sort_index())

B      50757
F    1499535
N     138214
P    3692350
Name: TP_STATUS_REDACAO, dtype: int64

--------------------------------------------------

1    3692350
2     138214
4    1550292
Name: TP_STATUS_REDACAO, dtype: int64


In [13]:
#Ajustando dados 6: NU_IDADE
df.loc[~(df['NU_IDADE'].isin(range(10, 101))), 'NU_IDADE'] = None

In [14]:
#Selecionando apenas os campos de interesse
df = df[campos].copy()
print(df.shape)

(5380856, 50)


In [15]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO
0,RESIDENCIA,3548708.0,SAO BERNARDO DO CAMPO,São Bernardo do Campo
1,RESIDENCIA,4125506.0,SAO JOSE DOS PINHAIS,São José dos Pinhais
2,RESIDENCIA,4314902.0,PORTO ALEGRE,Porto Alegre
3,RESIDENCIA,3300456.0,BELFORD ROXO,Belford Roxo
4,RESIDENCIA,2304400.0,FORTALEZA,Fortaleza
...,...,...,...,...
12697,PROVA,1303700.0,SANTO ANTONIO DO ICA,Santo Antônio do Içá
12698,PROVA,1301308.0,CODAJAS,Codajás
12699,PROVA,1301001.0,CARAUARI,Carauari
12700,PROVA,1303908.0,SAO PAULO DE OLIVENCA,São Paulo de Olivença


In [16]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
for c in ['RESIDENCIA','ESC','PROVA']:
    print(c, sum(~pd.Series(df[~df[f'CO_MUNICIPIO_{c}'].isna()][f'CO_MUNICIPIO_{c}'].unique()).isin(dict_municipios.keys()))==0,
          sep = ':    \t')

RESIDENCIA:    	True
ESC:    	True
PROVA:    	True


In [17]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
for c in ['RESIDENCIA','ESC','PROVA']: df[f'NO_MUNICIPIO_{c}'] = df[f'CO_MUNICIPIO_{c}'].map(dict_municipios)

In [18]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO


In [19]:
#Ajustando dados: SG_UF_RESIDENCIA/ESC/PROVA
check_ufs(df)

,COL,CO_MUNICIPIO,SG_MUNICIPIO,SIGLA_ESPERADA


In [20]:
#Verificando a consistência dos dados e sumarizando estatísticas descritivas
year_res_dict = {year:{'cols':df.shape[1], 'rows':df.shape[0], 'data':{}}}
tic()
for col in campos:
    res_prt, res_val = validate(df, col, year, get_values=True)
    print_results(col, res_prt); print()
    year_res_dict[year]['data'][col] = res_val
tac()

NU_INSCRICAO
-------------------------
Tipo:     	True
Tamanho:  	True
Unicidade:	True
Nulos:    	0

NU_ANO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

CO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

CO_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

SG_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	0

NU_IDADE
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	313

TP_SEXO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ESTADO_CIVIL
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_COR_RACA
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	13908

TP_ST_CONCLUSAO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0


In [21]:
#Armazenando as estatísticas descritivas sumarizadas em um arquivo json
with open(save_dir(year)['json'], 'w', encoding='utf-8') as fp: json.dump(year_res_dict, fp, cls=NpEncoder, ensure_ascii=False)

In [22]:
#Armazenando os dados do ano em um arquivo csv
tic()
df.to_csv(save_dir(year)['csv'], sep=';', header = True, index = False, encoding='latin')
tac()

Time passed: 0hour:1min:43sec


___

## 2010

In [5]:
#Obtendo informações do ano
year = 2010
tic()
df = pd.read_csv(get_dir(year), sep = ';', encoding = 'latin', dtype = 'O')
tac()

Time passed: 0hour:1min:1sec


In [6]:
num_campos = ['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'NU_IDADE', 'TP_ESTADO_CIVIL',
              'TP_COR_RACA', 'TP_ST_CONCLUSAO', 'TP_ENSINO', 'CO_ESCOLA', 'CO_MUNICIPIO_ESC', 'CO_UF_ESC',
              'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA',
              'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA', 'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL', 'IN_DEFICIT_ATENCAO',
              'IN_DISLEXIA', 'IN_GESTANTE', 'IN_LACTANTE', 'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'TP_PRESENCA_CN',
              'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
              'TP_LINGUA', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
              'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'] 

for c in df.columns: 
    df.loc[df[c]=='.', c] = None
    if c in num_campos:
        df[c] = pd.to_numeric(df[c])

In [7]:
#Ajustando dados 1: TP_ESTADO_CIVIL
#--------------------------------------------------------------------------------------
#NaN                                       → 0 Não informado 
#0 Solteiro(a)                             → 1 Solteiro(a)
#1 Casado(a)/Mora com companheiro(a)       → 2 Casado(a)/Mora com companheiro(a)
#2 Divorciado(a)/Desquitado(a)/Separado(a) → 3 Divorciado(a)/Desquitado(a)/Separado(a)
#3 Viúvo(a)                                → 4 Viúvo(a)
#--------------------------------------------------------------------------------------
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())
df['TP_ESTADO_CIVIL'] = df['TP_ESTADO_CIVIL'].map({0:1, 1:2, 2:3, 3:4})
df['TP_ESTADO_CIVIL'].fillna(0, inplace = True)
print();print(50*'-');print()
print(df['TP_ESTADO_CIVIL'].value_counts(dropna = False).sort_index())

0.0    3761624
1.0     745830
2.0      90136
3.0      14026
NaN      14478
Name: TP_ESTADO_CIVIL, dtype: int64

--------------------------------------------------

0.0      14478
1.0    3761624
2.0     745830
3.0      90136
4.0      14026
Name: TP_ESTADO_CIVIL, dtype: int64


In [8]:
#Ajustando dados 4: TP_ENSINO
#--------------------------------------------------------------------------------------
#1 Ensino Regular             → 1 Ensino Regular
#2 Ensino de Jovens e Adultos → 3 Educação de Jovens e Adultos
#3 Ensino Profissionalizante  → 4 Ensino Profissionalizante
#4 Ensino Especial            → 2 Educação Especial - Modalidade Substitutiva
#--------------------------------------------------------------------------------------
print(df['TP_ENSINO'].value_counts(dropna = False).sort_index())
df['TP_ENSINO'] = df['TP_ENSINO'].map({1:1, 2:3, 3:4, 4:2})
print();print(50*'-');print()
print(df['TP_ENSINO'].value_counts(dropna = False).sort_index())

1.0    3452374
2.0     401008
3.0     226641
4.0      15666
NaN     530405
Name: TP_ENSINO, dtype: int64

--------------------------------------------------

1.0    3452374
2.0      15666
3.0     401008
4.0     226641
NaN     530405
Name: TP_ENSINO, dtype: int64


In [9]:
#Ajustando dados 5: TP_STATUS_REDACAO
#--------------------------------------------------------------------------------------
#B Em Branco                       → 4 Em Branco
#D Desconsiderada                  → 7 Não atendimento ao tipo textual
#F Faltoso                         → 4 Em Branco
#N Anulada                         → 2 Anulada
#P Presente                        → 1 Sem problemas
#--------------------------------------------------------------------------------------
print(df['TP_STATUS_REDACAO'].value_counts(dropna = False).sort_index())
df['TP_STATUS_REDACAO'] = pd.to_numeric(df['TP_STATUS_REDACAO'].map({'B':4, 'D':7, 'F':4, 'N':2, 'P':1}))
print();print(50*'-');print()
print(df['TP_STATUS_REDACAO'].value_counts(dropna = False).sort_index())

B        43801
D       102420
F      1355081
N          614
P      3124176
NaN          2
Name: TP_STATUS_REDACAO, dtype: int64

--------------------------------------------------

1.0    3124176
2.0        614
4.0    1398882
7.0     102420
NaN          2
Name: TP_STATUS_REDACAO, dtype: int64


In [10]:
#Selecionando apenas os campos de interesse
df = df[campos].copy()
print(df.shape)

(4626094, 50)


In [11]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO
0,RESIDENCIA,3143302.0,MONTES CLAROS,Montes Claros
1,RESIDENCIA,1721000.0,PALMAS,Palmas
2,RESIDENCIA,1302603.0,MANAUS,Manaus
3,RESIDENCIA,3543907.0,RIO CLARO,Rio Claro
4,RESIDENCIA,3550308.0,SAO PAULO,São Paulo
...,...,...,...,...
12789,PROVA,4123857.0,SANTA MARIA DO OESTE,Santa Maria do Oeste
12790,PROVA,2904100.0,BOQUIRA,Boquira
12791,PROVA,1301605.0,FONTE BOA,Fonte Boa
12792,PROVA,2708303.0,SAO JOSE DA LAJE,São José da Laje


In [12]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
for c in ['RESIDENCIA','ESC','PROVA']:
    print(c, sum(~pd.Series(df[~df[f'CO_MUNICIPIO_{c}'].isna()][f'CO_MUNICIPIO_{c}'].unique()).isin(dict_municipios.keys()))==0,
          sep = ':    \t')

RESIDENCIA:    	True
ESC:    	True
PROVA:    	True


In [13]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
for c in ['RESIDENCIA','ESC','PROVA']: df[f'NO_MUNICIPIO_{c}'] = df[f'CO_MUNICIPIO_{c}'].map(dict_municipios)

In [14]:
#Ajustando dados 4: NO_MUNICIPIO_RESIDENCIA/ESC/PROVA
df_erros_municipio = check_municipios(df); display(df_erros_municipio)

,COL,CO_MUNICIPIO,NO_MUNICIPIO,NOME_ESPERADO


In [15]:
#Ajustando dados: SG_UF_RESIDENCIA/ESC/PROVA
check_ufs(df)

,COL,CO_MUNICIPIO,SG_MUNICIPIO,SIGLA_ESPERADA


In [16]:
#Verificando a consistência dos dados e sumarizando estatísticas descritivas
year_res_dict = {year:{'cols':df.shape[1], 'rows':df.shape[0], 'data':{}}}
tic()
for col in campos:
    res_prt, res_val = validate(df, col, year, get_values=True)
    print_results(col, res_prt); print()
    year_res_dict[year]['data'][col] = res_val
tac()

NU_INSCRICAO
-------------------------
Tipo:     	True
Tamanho:  	True
Unicidade:	True
Nulos:    	0

NU_ANO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

CO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	14473

NO_MUNICIPIO_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	14473

CO_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	14473

SG_UF_RESIDENCIA
-------------------------
Tipo:   	True
Tamanho:	True
Nulos:  	14473

NU_IDADE
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	332

TP_SEXO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_ESTADO_CIVIL
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	0

TP_COR_RACA
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	True
Nulos:  	14478

TP_ST_CONCLUSAO
-------------------------
Tipo:   	True
Tamanho:	True
Valores:	

In [17]:
#Armazenando as estatísticas descritivas sumarizadas em um arquivo json
with open(save_dir(year)['json'], 'w', encoding='utf-8') as fp: json.dump(year_res_dict, fp, cls=NpEncoder, ensure_ascii=False)

In [18]:
#Armazenando os dados do ano em um arquivo csv
tic()
df.to_csv(save_dir(year)['csv'], sep=';', header = True, index = False, encoding='latin')
tac()

Time passed: 0hour:1min:45sec


___

## CONSOLIDAÇÃO

In [67]:
#Consolidando os dados em um único arquivo csv
tic()
cnt = 0
with open(rood_dir+'MICRODADOS_ENEM_CONSOLIDADO_ALL.csv', 'a') as all_fl:
    all_fl.write(';'.join(campos)+'\n')
    for fl in [rood_dir+f'MICRODADOS_ENEM_CONSOLIDADO_{y}.csv' for y in range(2010, 2020)]:
        with open(fl, 'r') as f:
            for i, line in enumerate(f):
                if i == 0: 
                    pass
                else:
                    cnt+=1
                    all_fl.write(line)
tac()

Time passed: 0hour:3min:5sec


In [97]:
#Verificando o total de registros
cnt_conf = 0
dict_all = {}
for y in range(2010, 2020):
    with open(rood_dir+f'STATS_{y}.json', encoding='utf-8') as json_file:
        dict_all[y] = json.load(json_file)[str(y)]
        cnt_conf = cnt_conf + int(dict_all[y]['rows'])
print(f'Total de registros ({cnt:,}):'.replace(',','.'),cnt == cnt_conf)

Total de registros (65.407.978): True


In [90]:
#Consolidando as estatísticas sumarizadas em um único arquivo json
with open(rood_dir+'\\STATS_ALL.json', 'w', encoding='utf-8') as fp: json.dump(dict_all, fp, cls=NpEncoder, ensure_ascii=False)

In [120]:
#Conferindo o totais de nulos por campo e ano
null_dict = {c:{} for c in range(2010,2020)}
for y in range(2010, 2020):
    for c in campos:
        null_dict[y][c] = dict_all[y]['data'][c]['nul']
pd.DataFrame(null_dict)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
NU_INSCRICAO,0,0,0,0,0,0,0,0,0,0
NU_ANO,0,0,0,0,0,0,0,0,0,0
CO_MUNICIPIO_RESIDENCIA,14473,0,0,0,0,0,0,0,0,0
NO_MUNICIPIO_RESIDENCIA,14473,0,0,0,0,0,0,0,0,0
CO_UF_RESIDENCIA,14473,0,0,0,0,0,0,0,0,0
SG_UF_RESIDENCIA,14473,0,0,0,0,0,0,0,0,0
NU_IDADE,332,313,160,243,227,75,102,101,97,69
TP_SEXO,0,0,0,0,0,0,0,0,0,0
TP_ESTADO_CIVIL,0,0,0,0,0,0,0,0,0,0
TP_COR_RACA,14478,13908,0,0,0,0,0,0,0,0
